In [1]:
import geopandas as gpd
import ipywidgets as widgets
import pandas as pd

from IPython.display import Markdown, HTML

import setup_corridors_stats
from create_parallel_corridors import IMG_PATH, DATA_PATH
from shared_utils import map_utils, geography_utils
from shared_utils import calitp_color_palette as cp

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0414 18:52:50.461524330    1378 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0414 18:52:52.708019355    1378 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
display(HTML("<h1>Parallel / Intersecting Bus Routes to the State Highway Network </h1>"))

In [3]:
gdf = gpd.read_parquet(f"{DATA_PATH}parallel_or_intersecting.parquet")
highways= gpd.read_parquet(f"{DATA_PATH}highways.parquet")

In [4]:
operator_hwys = (gdf[["itp_id", "Route"]][gdf.Route.notna()]
                 .drop_duplicates()
                 .astype(int)
                )

operator_hwys = (operator_hwys.groupby("itp_id")["Route"]
                 .apply(lambda x: x.tolist())
                 .reset_index()
                 .rename(columns = {"Route": "hwy_list"})
                )

In [5]:
# Move aggregations to be done ahead of time, before dropdown
rename_dict = {
    "itp_id": "ITP ID",
    "pct_parallel": "% parallel",
    "unique_route_id": "# Unique Bus Routes",
    "num_parallel": "# Parallel Bus Routes",
}

operator_stats = (setup_corridors_stats.aggregate(gdf, by="operator")
                  .sort_values("pct_parallel", ascending=False)
                  .reset_index(drop=True)
                  .merge(operator_hwys, on = "itp_id")
                 )

hwy_stats = (setup_corridors_stats.aggregate(gdf, by="highway")
             .sort_values("pct_parallel", ascending=False)
             .reset_index(drop=True)
            )    

In [6]:
def display_operator_stats(operator_stats, select_col = "itp_id", operator_name = 182, 
                           base_df = gdf):
    # Style table
    operator_stats = operator_stats[operator_stats[select_col]==operator_name]
    
    operator_table = (operator_stats.sort_values("pct_parallel", ascending=False)
                      .drop(columns = "hwy_list")
                      .rename(columns = rename_dict)
                      #.style({'% parallel': '{:,.1%}'.format,})
    )
    
    hwy_df = highways[highways.Route.isin(operator_stats.hwy_list)]
    
    display(HTML(f"<h2> Summary Stats for ITP ID: {operator_name}</h2>"))
    display(operator_table)
    
    # ugh, add gdf as additional arg? 
    make_transit_map(gdf[select_col]==operator_name, hwy_df)

In [7]:
def make_transit_map(operator_df, hwy_df):
    ## Add a map to show the operator's routes and highways from C1
    
    def data_to_plot(df):
        keep_cols = ["itp_id", "route_id", 
                     "Route", "County", "District", "RouteType",
                     "pct_route", "pct_highway", "parallel",
                     "geometry"
                    ]
        df = df[keep_cols].reset_index(drop=True)
        df = df.assign(
            geometry = df.geometry.buffer(200).simplify(tolerance=100),
        )

        # Use simplify to make gdf smaller
        # folium map is creating too large of an HTML file to check in

        return df

    to_map = data_to_plot(operator_df)
    
    # Set various components for map
    hwys_popup_dict = {
        "Route": "Highway Route",
        "RouteType": "Route Type",
        "County": "County"   
    }

    transit_popup_dict = {
        "itp_id": "Operator ITP ID",
        "route_id": "Route ID",
        "pct_route": "% overlapping route",
        "pct_highway": "% overlapping highway",
    }

    hwys_color = branca.colormap.StepColormap(
        colors=["black", "gray"],
    )

    colorscale = branca.colormap.StepColormap(
        colors=[
            cp.CALITP_CATEGORY_BRIGHT_COLORS[0], #blue
            cp.CALITP_CATEGORY_BRIGHT_COLORS[1] # orange
        ],
    )
    
    # Instead of using county centroid, calculate centroid from transit_df
    # Otherwise, it's too zoomed out from where transit routes are
    transit_centroid = (to_map
                        .to_crs(geography_utils.WGS84).geometry.centroid
                        .iloc[0]
                       )

    LAYERS_DICT = {
        "Highways": {"df": hwy_df,
            "plot_col": "Route",
            "popup_dict": hwys_popup_dict, 
            "tooltip_dict": hwys_popup_dict,
            "colorscale": hwys_color,
        },
        "Transit Routes": {"df": to_map,
            "plot_col": "parallel",
            "popup_dict": transit_popup_dict, 
            "tooltip_dict": transit_popup_dict,
            "colorscale": colorscale,
        },
    }
    
    LEGEND_URL = (
        "https://github.com/cal-itp/data-analyses/raw/"
        "main/bus_service_increase/"
        "img/legend_intersecting_parallel.png"
    )
    
    LEGEND_DICT = {
        "legend_url": LEGEND_URL,
        "legend_bottom": 85,
        "legend_left": 5,
    }
     
    
    fig = map_utils.make_folium_multiple_layers_map(
        LAYERS_DICT,
        fig_width = 700, fig_height = 700, 
        zoom=11, 
        centroid = [round(transit_centroid.y,2), 
                    round(transit_centroid.x, 2)], 
        title=f"Parallel vs Intersecting Lines for {to_map.itp_id.iloc[0]}",
        legend_dict = LEGEND_DICT
    )
    
    display(fig)
    #fig.save(f"{IMG_PATH}parallel_{operator_name}.html")
    #print(f"{operator_name} map saved")

In [8]:
def display_highway_stats(hwy_stats, select_col = "Route", hwy_name = 5, base_df = gdf):
    # Subset for highways
    hwy_stats = hwy_stats[hwy_stats[select_col]==hwy_name]
    hwy_table = (hwy_stats.sort_values("pct_parallel", ascending=False)
                 .rename(columns = rename_dict)
                 .reset_index(drop=True)
                 #.style({'% parallel': '{:,.1%}'.format,})
                )    
    
    display(HTML(f"<h3>Summary Stats for Highway Route: {hwy_name} </h3>"))
    display(hwy_table)
    

In [9]:
## Think about how to wrap above interactive widget into a function
# Be able to select agency, county, district, etc
def interactive_widget(df, select_col):
    dropdown_labels = {
        "itp_id": "ITP ID",
        "Route": "Hwy Route",
    }
    
    dropdown = widgets.Dropdown(
        description=f"{dropdown_labels[select_col]}",
        options=df[select_col].sort_values().unique().tolist(),
    )
    output = widgets.Output()

    display(dropdown)
    display(output)

    def on_selection(*args):
        output.clear_output()
        with output:
            if select_col=="Route":
                display_highway_stats(hwy_stats, select_col, dropdown.value)
            elif select_col=="itp_id":
                display_operator_stats(operator_stats, select_col, dropdown.value)

    dropdown.observe(on_selection, names="value")
    on_selection()

In [10]:
interactive_widget(operator_stats, "itp_id")

Dropdown(description='ITP ID', options=(4, 6, 10, 11, 13, 14, 15, 16, 17, 18, 21, 23, 29, 30, 33, 34, 35, 36, …

Output()

In [11]:
interactive_widget(hwy_stats, "Route")

Dropdown(description='Hwy Route', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, …

Output()